In [1]:
import pandas as pd
import sys
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
from sklearn.model_selection import train_test_split
from statsmodels.tools import add_constant
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
import statsmodels.api as sm
import pickle
import os
from sklearn import preprocessing
sns.set(style="ticks")
%matplotlib inline

am = pd.read_csv("boston_NO_LU_and_weather.csv")
a = am.loc[:, ["Latitude", "Longitude","Sample Collection Start Time", 'Measure Value', 'Cropland',
       'Golf Course', 'Saltwater Sandy Beach', 'cemetary','commercial','Parameter Name','crop_land', 
           'forest', 'high_density_residential', 'industrial',
       'low_density_residential', 'marina', 'medium_density_residential',
       'mining', 'open_land', 'recreational', 'transitional', 'transportation',
           'urban_public/institutional', 'waste', 'water', 'wetland', 'temp_c', 'wind_kph', 'UV']]
'''Rename columns for a consistent naming format'''
def rename_again(df):
    df=df.rename(columns = {'Saltwater Sandy Beach':'saltwater_sandybeach'})
    df=df.rename(columns = {'urban_public/institutional':'urban_public_institution'})
    return df
a=rename_again(a)

In [2]:
"""Delete Variables that do not contribute to ppm"""
#Deletions only need to be made for PM
del a["saltwater_sandybeach"]
del a["recreational"]
del a["marina"]
del a["mining"]
del a["waste"]
del a["cemetary"]
del a["Cropland"]
del a["Golf Course"]
del a["high_density_residential"]
del a["low_density_residential"]
#del am["not_assigned"]

In [3]:
#renames weather correctly
a.columns = ['Latitude', 'Longitude', 'Sample Collection Start Time',
       'ppm', 'commercial', 'Parameter Name', 'crop_land', 'forest',
       'industrial', 'medium_density_residential', 'open_land', 'transitional',
       'transportation', 'urban_public_institution', 'water', 'wetland',
       'outdoor_temperature', 'wind_speed_resultant', 'solar_radiation']

In [4]:
pm = pd.read_csv("finished_no2weather.csv")
del pm["Sulfation rate"]
del pm["Dew Point"]

In [5]:
def parse_string(str_edit):
    str_edit = str_edit.replace('[', '')
    str_edit = str_edit.replace(' ', '')
#     str_edit = str_edit.replace(' ', ',')
    str_edit = str_edit.replace(']', '')
    list = str_edit.split('-')
    return np.float64(list[0]), (-1.*np.float64(list[1]))
parse_string(pm["Site_keys_x"][0])[0]

34.803421

In [6]:
lat = []
long = []
for i in pm["Site_keys_x"]:
    lat.append(parse_string(i)[0])
    long.append(parse_string(i)[1])
pm["Latitude"] = lat
pm["Longitude"] = long
pm['Sample Collection Start Time'] = 1.00
del pm["Site_keys_x"]

In [7]:
pm.columns

Index(['Parameter Name', 'Arithmetic Mean', 'forest', 'open_land', 'water',
       'wetland', 'transitional', 'urban_public_institution', 'commercial',
       'transportation', 'crop_land', 'medium_density_residential',
       'industrial', 'outdoor_temperature', 'solar_radiation',
       'wind_speed_resultant', 'Latitude', 'Longitude',
       'Sample Collection Start Time'],
      dtype='object')

In [12]:
#reorders columns directly
cols = ['Parameter Name', 'Latitude', 'Longitude', "Sample Collection Start Time", 'ppm', 'forest',
       'open_land', 'water', 'wetland', 'transitional',
       'urban_public_institution', 'commercial', 'transportation', 'crop_land',
       'medium_density_residential', 'industrial',
       'outdoor_temperature', 'solar_radiation', 
       'wind_speed_resultant']
col = ['Parameter Name', 'ppm', 'forest', 'open_land', 'water',
       'wetland', 'transitional', 'urban_public_institution', 'commercial',
       'transportation', 'crop_land', 'medium_density_residential',
       'industrial', 'outdoor_temperature', 'solar_radiation',
       'wind_speed_resultant', 'Latitude', 'Longitude',
       'Sample Collection Start Time']
a = a[cols]
a.to_csv("boston_only_no2.csv", index=False)
pd.read_csv("boston_only_no2.csv")

,Parameter Name,Latitude,Longitude,Sample Collection Start Time,ppm,forest,open_land,water,wetland,transitional,urban_public_institution,commercial,transportation,crop_land,medium_density_residential,industrial,outdoor_temperature,solar_radiation,wind_speed_resultant
0,Nitrogen Dioxide,42.348873,-71.097163,10.0,0.010,0.0,0.00,0.0,0.0,0.00,0.11,0.59,0.02,0,0.0,0.00,23.77619,1.059524,8.497024
1,Nitrogen Dioxide,42.348873,-71.097163,11.0,0.007,0.0,0.00,0.0,0.0,0.00,0.11,0.59,0.02,0,0.0,0.00,23.77619,1.059524,8.497024
2,Nitrogen Dioxide,42.348873,-71.097163,12.0,0.006,0.0,0.00,0.0,0.0,0.00,0.11,0.59,0.02,0,0.0,0.00,23.77619,1.059524,8.497024
3,Nitrogen Dioxide,42.348873,-71.097163,13.0,0.009,0.0,0.00,0.0,0.0,0.00,0.11,0.59,0.02,0,0.0,0.00,23.77619,1.059524,8.497024
4,Nitrogen Dioxide,42.348873,-71.097163,14.0,0.006,0.0,0.00,0.0,0.0,0.00,0.11,0.59,0.02,0,0.0,0.00,23.77619,1.059524,8.497024
5,Nitrogen Dioxide,42.348873,-71.097163,15.0,0.007,0.0,0.00,0.0,0.0,0.00,0.11,0.59,0.02,0,0.0,0.00,23.77619,1.059524,8.497024
6,Nitrogen Dioxide,42.348873,-71.097163,16.0,0.016,0.0,0.00,0.0,0.0,0.00,0.11,0.59,0.02,0,0.0,0.00,23.77619,1.059524,8.497024
7,Nitrogen Dioxide,42.348873,-71.097163,17.0,0.008,0.0,0.00,0.0,0.0,0.00,0.11,0.59,0.02,0,0.0,0.00,23.77619,1.059524,8.497024
8,Nitrogen Dioxide,42.348873,-71.097163,18.0,0.008,0.0,0.00,0.0,0.0,0.00,0.11,0.59,0.02,0,0.0,0.00,23.77619,1.059524,8.497024
9,Nitrogen Dioxide,42.348873,-71.097163,19.0,0.009,0.0,0.00,0.0,0.0,0.00,0.11,0.59,0.02,0,0.0,0.00,23.77619,1.059524,8.497024


In [76]:
pm.columns = col
pm = pm[cols]
w = a.append(pm)
w.to_csv("merged_no2weather.csv", index=False)
pd.read_csv("merged_no2weather.csv")

,Parameter Name,Latitude,Longitude,Sample Collection Start Time,ppm,forest,open_land,water,wetland,transitional,urban_public_institution,commercial,transportation,crop_land,medium_density_residential,industrial,outdoor_temperature,solar_radiation,wind_speed_resultant
0,Nitrogen Dioxide,42.348873,-71.097163,10.0,0.010000,0.000000,0.00,0.000000,0.0,0.00000,0.110000,0.590000,0.020000,0.000000,0.000000,0.000000,23.776190,1.059524,8.497024
1,Nitrogen Dioxide,42.348873,-71.097163,11.0,0.007000,0.000000,0.00,0.000000,0.0,0.00000,0.110000,0.590000,0.020000,0.000000,0.000000,0.000000,23.776190,1.059524,8.497024
2,Nitrogen Dioxide,42.348873,-71.097163,12.0,0.006000,0.000000,0.00,0.000000,0.0,0.00000,0.110000,0.590000,0.020000,0.000000,0.000000,0.000000,23.776190,1.059524,8.497024
3,Nitrogen Dioxide,42.348873,-71.097163,13.0,0.009000,0.000000,0.00,0.000000,0.0,0.00000,0.110000,0.590000,0.020000,0.000000,0.000000,0.000000,23.776190,1.059524,8.497024
4,Nitrogen Dioxide,42.348873,-71.097163,14.0,0.006000,0.000000,0.00,0.000000,0.0,0.00000,0.110000,0.590000,0.020000,0.000000,0.000000,0.000000,23.776190,1.059524,8.497024
5,Nitrogen Dioxide,42.348873,-71.097163,15.0,0.007000,0.000000,0.00,0.000000,0.0,0.00000,0.110000,0.590000,0.020000,0.000000,0.000000,0.000000,23.776190,1.059524,8.497024
6,Nitrogen Dioxide,42.348873,-71.097163,16.0,0.016000,0.000000,0.00,0.000000,0.0,0.00000,0.110000,0.590000,0.020000,0.000000,0.000000,0.000000,23.776190,1.059524,8.497024
7,Nitrogen Dioxide,42.348873,-71.097163,17.0,0.008000,0.000000,0.00,0.000000,0.0,0.00000,0.110000,0.590000,0.020000,0.000000,0.000000,0.000000,23.776190,1.059524,8.497024
8,Nitrogen Dioxide,42.348873,-71.097163,18.0,0.008000,0.000000,0.00,0.000000,0.0,0.00000,0.110000,0.590000,0.020000,0.000000,0.000000,0.000000,23.776190,1.059524,8.497024
9,Nitrogen Dioxide,42.348873,-71.097163,19.0,0.009000,0.000000,0.00,0.000000,0.0,0.00000,0.110000,0.590000,0.020000,0.000000,0.000000,0.000000,23.776190,1.059524,8.497024
